In [1]:
# ------------------------------------------------------------------------
# Author: Euihyun Choi, Structural Engineering Intern. Date: July 20, 2022
# AECOM Technical Services Inc. 
# Project: Pilarcitos Dam and Reservoir Improvement Project
# Task: Preliminary design of Labyrinth weir
# ------------------------------------------------------------------------

import os
import numpy as np
import re
import matplotlib.pyplot as plt
import imageio
import random
from shapely.geometry import LineString, Point, Polygon

# Standard Units
ft = 1
kips = 1

# Dependant Units
inch = ft/12
lb = kips/1000
pcf = lb/ft**3
ksi = kips/inch/inch
psi = lb/inch/inch

# Material Properties
gamma_conc = 150 * pcf # Concrete density
gamma_w = 62.4 * pcf # Water unit weight
fc = 4 * ksi
fy = 60 * ksi

# Geotech Input
gamma_soil = 135 * pcf # Soil unit weight
Ka = 0.277
Ka_eq = 0.4
Kp = 3.385
mu = 0.5
c = 0
PGA = 0.5
kh = 0.7

# Water Level
El_NWS = 696.5*ft # Normal Water Surface Elevation
El_MWS = 704*ft # Maximum Water Surface Elevation

L_weir = 450*ft
El_weir_crest = 696.5*ft
level = [705, 700, 695, 690, 685, 680, 675, 670, 665, 660, 655]

ts = 60*inch
tk = 36*inch
hk = 59.8*inch
hw = 13.5*ft
hw_b = 15*ft
tw_b = 45*inch

k1 = 0.1
k2 = 1000
k3 = 120
N_mult = 0
%run -i "data_read.ipynb"
PP = 0
fign = -1
fignames = []
def CrossOver(Gen, N, CO_rate, RM_rate, rand_bound, params_ub, params_lb):
    offsprings = []
    for i in range(N):
        parent1 = np.random.randint(0,len(Gen))
        parent2 = np.random.randint(0,len(Gen))
        p = np.random.uniform(0,1)
        # random searching near the best candidate:
        if p <= RM_rate:
            parent1 = 0
            candidate = Gen[parent1] + np.multiply(np.random.uniform(-rand_bound, rand_bound, len(Gen[parent1])), Gen[parent1])
            for i in range(len(params_ub)):
                if candidate[i] > params_ub[i]:
                    candidate[i] = params_ub[i]
                if candidate[i] < params_ub[i]:
                    candidate[i] = params_lb[i]
        else:
            while parent1 == parent2:
                parent2 = np.random.randint(0,len(Gen))
            multip = []
            for i in range(len(Gen[parent1])):
                if i < len(Gen[parent1])*CO_rate:
                    multip.append(random.uniform(0,1))
                else:
                    multip.append(random.randint(0,1))
            random.shuffle(multip)
            candidate = Gen[parent1] + np.multiply(multip, Gen[parent2]-Gen[parent1])
        offsprings.append(candidate)
    return offsprings
Gen_num = 20
popul = 50
survive_rate = 0.4
offspring_rate = 0.4
CO_rate = 0.7
RM_rate = 0.2
rand_bound = 0.005
Neval = Gen_num *(1-survive_rate)*popul+survive_rate*popul

params_lb = np.array([80, 50, 85/180*np.pi, 40, 40, 0, 0, 2, 0])
params_ub = np.array([120, 110, 110/180*np.pi, 80, 80, 10, 8, 6, 20])

Gen = []

for k in range(popul):
    flag = 0
    while flag == 0:
        set_temp = params_lb + np.multiply(np.random.uniform(0,1,len(params_ub)),params_ub-params_lb)
        flag = 1
        L1 = set_temp[0]
        L2 = set_temp[1]
        theta = set_temp[2]
        W = set_temp[3]
        B = set_temp[4]
        A1 = set_temp[5]
        A2 = set_temp[6]
        tw = set_temp[7]
        We = set_temp[8]
        L3 = np.sqrt(L1**2+L2**2-2*L1*L2*np.cos(theta))
        W_min = L3/2 - We
        w = (W/2 - (A1+A2))/2
        L = B-tw
        lw = (L**2+w**2)**0.5
        Lw = (lw*2 + A1 + A2)*4 + 2*We
        if W_min < 0:
            flag = 0
        if W < W_min:
            flag = 0
        if Lw < L_weir:
            flag = 0
    Gen.append(set_temp)

In [2]:
evalnum = 0
best = [0,0,0,0,10000000000]
# recorder_params = []
recorder_objs = []
Best = Gen[0]
for k in range(popul):
    candid = Gen[k]
    L1 = candid[0]
    L2 = candid[1]
    theta = candid[2]
    W = candid[3]
    B = candid[4]
    A1 = candid[5]
    A2 = candid[6]
    tw = candid[7]
    We = candid[8]
    fign += 1

    %run -i "obj_func.ipynb"

def error_sort(list):
    return list[len(params_ub)]
Gen.sort(key=error_sort)
Gen_best = [Gen[0]]
Nsurvive = round(popul*survive_rate)
for N in range(Gen_num-1):
    del Gen[Nsurvive:popul]
    offsprings = CrossOver(Gen, round(popul*offspring_rate), CO_rate, RM_rate, rand_bound, params_ub, params_lb)
    eval_N = popul - Nsurvive
    for k in range(round(popul*offspring_rate)):
        offsprings[k] = np.delete(offsprings[k], -1)
        set_temp = offsprings[k]
        flag = 1
        L1 = set_temp[0]
        L2 = set_temp[1]
        theta = set_temp[2]
        W = set_temp[3]
        B = set_temp[4]
        A1 = set_temp[5]
        A2 = set_temp[6]
        tw = set_temp[7]
        We = set_temp[8]
        L3 = np.sqrt(L1**2+L2**2-2*L1*L2*np.cos(theta))
        W_min = L3/2 - We
        w = (W/2 - (A1+A2))/2
        L = B-tw
        lw = (L**2+w**2)**0.5
        Lw = (lw*2 + A1 + A2)*4 + 2*We
        if W_min < 0:
            flag = 0
        if W < W_min:
            flag = 0
        if Lw < L_weir:
            flag = 0
        while flag == 0:
            set_temp = params_lb + np.multiply(np.random.uniform(0,1,len(params_ub)),params_ub-params_lb)
            flag = 1
            L1 = set_temp[0]
            L2 = set_temp[1]
            theta = set_temp[2]
            W = set_temp[3]
            B = set_temp[4]
            A1 = set_temp[5]
            A2 = set_temp[6]
            tw = set_temp[7]
            We = set_temp[8]
            L3 = np.sqrt(L1**2+L2**2-2*L1*L2*np.cos(theta))
            W_min = L3/2 - We
            w = (W/2 - (A1+A2))/2
            L = B-tw
            lw = (L**2+w**2)**0.5
            Lw = (lw*2 + A1 + A2)*4 + 2*We
            if W_min < 0:
                flag = 0
            if W < W_min:
                flag = 0
            if Lw < L_weir:
                flag = 0

        Gen.append(set_temp)
    for k in range(popul - Nsurvive - round(popul*offspring_rate)):
        flag = 0
        while flag == 0:
            set_temp = params_lb + np.multiply(np.random.uniform(0,1,len(params_ub)),params_ub-params_lb)
            flag = 1
            L1 = set_temp[0]
            L2 = set_temp[1]
            theta = set_temp[2]
            W = set_temp[3]
            B = set_temp[4]
            A1 = set_temp[5]
            A2 = set_temp[6]
            tw = set_temp[7]
            We = set_temp[8]
            L3 = np.sqrt(L1**2+L2**2-2*L1*L2*np.cos(theta))
            W_min = L3/2 - We
            w = (W/2 - (A1+A2))/2
            L = B-tw
            lw = (L**2+w**2)**0.5
            Lw = (lw*2 + A1 + A2)*4 + 2*We
            if W_min < 0:
                flag = 0
            if W < W_min:
                flag = 0
            if Lw < L_weir:
                flag = 0
        Gen.append(set_temp)
    
    
    for k in range(eval_N):
        candid = Gen[k+popul-eval_N]
        L1 = candid[0]
        L2 = candid[1]
        theta = candid[2]
        W = candid[3]
        B = candid[4]
        A1 = candid[5]
        A2 = candid[6]
        tw = candid[7]
        We = candid[8]
        fign += 1
        PP = 1
        %run -i "obj_func.ipynb"

    Gen.sort(key=error_sort)


# build gif
with imageio.get_writer('weir.gif', mode='I') as writer:
    for figname in fignames:
        image = imageio.imread(figname)
        writer.append_data(image)
        
# Remove files
for figname in set(fignames):
    os.remove(figname)